In [1]:
import warnings
from typing import List, Optional, Tuple, Union

import torch
from torch import nn


class MLP(torch.nn.Sequential):
    """MLP Module.

    Parameters
    ----------
    in_features: int
        Features (model input) dimension.
    out_features: int = 1
        Prediction (model output) dimension.
    hidden: Optional[List[int]] = None
        Dimension of hidden layer(s).
    dropout: Optional[List[float]] = None
        Dropout rate(s).
    activation: Optional[torch.nn.Module] = torch.nn.Sigmoid
        MLP activation.
    bias: bool = True
        Add bias to MLP hidden layers.

    Raises
    ------
    ValueError
        If ``hidden`` and ``dropout`` do not share the same length.
    """

    def __init__(
            self,
            in_features: int,
            out_features: int,
            hidden: Optional[List[int]] = None,
            dropout: Optional[List[float]] = None,
            activation: Optional[torch.nn.Module] = torch.nn.Sigmoid(),
            bias: bool = True,
    ):
        if dropout is not None:
            if hidden is not None:
                assert len(hidden) == len(
                    dropout
                ), "hidden and dropout must have the same length"
            else:
                raise ValueError(
                    "hidden must have a value and have the same length as dropout if dropout is given."
                )

        d_model = in_features
        layers = []

        if hidden is not None:
            for i, h in enumerate(hidden):
                seq = [torch.nn.Linear(d_model, h, bias=bias)]
                d_model = h

                if activation is not None:
                    seq.append(activation)

                if dropout is not None:
                    seq.append(torch.nn.Dropout(dropout[i]))

                layers.append(torch.nn.Sequential(*seq))

        layers.append(torch.nn.Linear(d_model, out_features))

        super(MLP, self).__init__(*layers)


class MaskedLinear(torch.nn.Linear):
    """
    Linear layer to be applied tile wise.
    This layer can be used in combination with a mask
    to prevent padding tiles from influencing the values of a subsequent
    activation.
    Example:
        >>> module = Linear(in_features=128, out_features=1) # With Linear
        >>> out = module(slide)
        >>> wrong_value = torch.sigmoid(out) # Value is influenced by padding
        >>> module = MaskedLinear(in_features=128, out_features=1, mask_value='-inf') # With MaskedLinear
        >>> out = module(slide, mask) # Padding now has the '-inf' value
        >>> correct_value = torch.sigmoid(out) # Value is not influenced by padding as sigmoid('-inf') = 0
    Parameters
    ----------
    in_features: int
        size of each input sample
    out_features: int
        size of each output sample
    mask_value: Union[str, int]
        value to give to the mask
    bias: bool = True
        If set to ``False``, the layer will not learn an additive bias.
    """

    def __init__(
            self,
            in_features: int,
            out_features: int,
            mask_value: Union[str, float],
            bias: bool = True,
    ):
        super(MaskedLinear, self).__init__(
            in_features=in_features, out_features=out_features, bias=bias
        )
        self.mask_value = mask_value

    def forward(
            self, x: torch.Tensor, mask: Optional[torch.BoolTensor] = None
    ):  # pylint: disable=arguments-renamed
        """Forward pass.

        Parameters
        ----------
        x: torch.Tensor
            Input tensor, shape (B, SEQ_LEN, IN_FEATURES).
        mask: Optional[torch.BoolTensor] = None
            True for values that were padded, shape (B, SEQ_LEN, 1),

        Returns
        -------
        x: torch.Tensor
            (B, SEQ_LEN, OUT_FEATURES)
        """
        x = super(MaskedLinear, self).forward(x)
        if mask is not None:
            x = x.masked_fill(mask, float(self.mask_value))
        return x

    def extra_repr(self):
        return (
            f"in_features={self.in_features}, out_features={self.out_features}, "
            f"mask_value={self.mask_value}, bias={self.bias is not None}"
        )


class TilesMLP(torch.nn.Module):
    """MLP to be applied to tiles to compute scores.
    This module can be used in combination of a mask
    to prevent padding from influencing the scores values.
    Parameters
    ----------
    in_features: int
        size of each input sample
    out_features: int
        size of each output sample
    hidden: Optional[List[int]] = None
        Number of hidden layers and their respective number of features.
    bias: bool = True
        If set to ``False``, the layer will not learn an additive bias.
    activation: torch.nn.Module = torch.nn.Sigmoid()
        MLP activation function
    dropout: Optional[torch.nn.Module] = None
        Optional dropout module. Will be interlaced with the linear layers.
    """

    def __init__(
            self,
            in_features: int,
            out_features: int = 1,
            hidden: Optional[List[int]] = None,
            bias: bool = True,
            activation: torch.nn.Module = torch.nn.Sigmoid(),
            dropout: Optional[torch.nn.Module] = None,
    ):
        super(TilesMLP, self).__init__()

        self.hidden_layers = torch.nn.ModuleList()
        if hidden is not None:
            for h in hidden:
                self.hidden_layers.append(
                    MaskedLinear(in_features, h, bias=bias, mask_value="-inf")
                )
                self.hidden_layers.append(activation)
                if dropout:
                    self.hidden_layers.append(dropout)
                in_features = h

        self.hidden_layers.append(
            torch.nn.Linear(in_features, out_features, bias=bias)
        )

    def forward(
            self, x: torch.Tensor, mask: Optional[torch.BoolTensor] = None
    ):
        """Forward pass.

        Parameters
        ----------
        x: torch.Tensor
            (B, N_TILES, IN_FEATURES)
        mask: Optional[torch.BoolTensor] = None
            (B, N_TILES), True for values that were padded.

        Returns
        -------
        x: torch.Tensor
            (B, N_TILES, OUT_FEATURES)
        """
        for layer in self.hidden_layers:
            # print(x.size())
            if isinstance(layer, MaskedLinear):
                x = layer(x, mask)
            else:
                x = layer(x)

        return x


class ExtremeLayer(torch.nn.Module):
    """Extreme layer.
    Returns concatenation of n_top top tiles and n_bottom bottom tiles
    .. warning::
        If top tiles or bottom tiles is superior to the true number of
        tiles in the input then padded tiles will be selected and their value
        will be 0.
    Parameters
    ----------
    n_top: Optional[int] = None
        Number of top tiles to select
    n_bottom: Optional[int] = None
        Number of bottom tiles to select
    dim: int = 1
        Dimension to select top/bottom tiles from
    return_indices: bool = False
        Whether to return the indices of the extreme tiles

    Raises
    ------
    ValueError
        If ``n_top`` and ``n_bottom`` are set to ``None`` or both are 0.
    """

    def __init__(
            self,
            n_top: Optional[int] = None,
            n_bottom: Optional[int] = None,
            dim: int = 1,
            return_indices: bool = False,
    ):
        super(ExtremeLayer, self).__init__()

        if not (n_top is not None or n_bottom is not None):
            raise ValueError("one of n_top or n_bottom must have a value.")

        if not (
                (n_top is not None and n_top > 0)
                or (n_bottom is not None and n_bottom > 0)
        ):
            raise ValueError("one of n_top or n_bottom must have a value > 0.")

        self.n_top = n_top
        self.n_bottom = n_bottom
        self.dim = dim
        self.return_indices = return_indices

    def forward(
            self, x: torch.Tensor, mask: Optional[torch.BoolTensor] = None
    ) -> Union[torch.Tensor, Tuple[torch.Tensor, torch.Tensor]]:
        """Forward pass.
        Parameters
        ----------
        x: torch.Tensor
            Input tensor, shape (B, N_TILES, IN_FEATURES).
        mask: Optional[torch.BoolTensor]
            True for values that were padded, shape (B, N_TILES, 1).

        Warnings
        --------
        If top tiles or bottom tiles is superior to the true number of tiles in
        the input then padded tiles will be selected and their value will be 0.

        Returns
        -------
        values: torch.Tensor
            Extreme tiles, shape (B, N_TOP + N_BOTTOM).
        indices: torch.Tensor
            If ``self.return_indices=True``, return extreme tiles' indices.
        """

        if (
                self.n_top
                and self.n_bottom
                and ((self.n_top + self.n_bottom) > x.shape[self.dim])
        ):
            warnings.warn(
                f"Sum of tops is larger than the input tensor shape for dimension {self.dim}: "
                f"{self.n_top + self.n_bottom} > {x.shape[self.dim]}. "
                f"Values will appear twice (in top and in bottom)"
            )

        top, bottom = None, None
        top_idx, bottom_idx = None, None
        if mask is not None:
            if self.n_top:
                top, top_idx = x.masked_fill(mask, float("-inf")).topk(
                    k=self.n_top, sorted=True, dim=self.dim
                )
                top_mask = top.eq(float("-inf"))
                if top_mask.any():
                    warnings.warn(
                        "The top tiles contain masked values, they will be set to zero."
                    )
                    top[top_mask] = 0

            if self.n_bottom:
                bottom, bottom_idx = x.masked_fill(mask, float("inf")).topk(
                    k=self.n_bottom, largest=False, sorted=True, dim=self.dim
                )
                bottom_mask = bottom.eq(float("inf"))
                if bottom_mask.any():
                    warnings.warn(
                        "The bottom tiles contain masked values, they will be set to zero."
                    )
                    bottom[bottom_mask] = 0
        else:
            if self.n_top:
                top, top_idx = x.topk(k=self.n_top, sorted=True, dim=self.dim)
            if self.n_bottom:
                bottom, bottom_idx = x.topk(
                    k=self.n_bottom, largest=False, sorted=True, dim=self.dim
                )

        if top is not None and bottom is not None:
            values = torch.cat([top, bottom], dim=self.dim)
            indices = torch.cat([top_idx, bottom_idx], dim=self.dim)
        elif top is not None:
            values = top
            indices = top_idx
        elif bottom is not None:
            values = bottom
            indices = bottom_idx
        else:
            raise ValueError

        if self.return_indices:
            return values, indices
        else:
            return values

    def extra_repr(self) -> str:
        """Format representation."""
        return f"n_top={self.n_top}, n_bottom={self.n_bottom}"


class Chowder(nn.Module):
    """Chowder MIL model (See [1]_).

    Example:
        >>> module = Chowder(in_features=128, out_features=1, n_top=5, n_bottom=5)
        >>> logits, extreme_scores = module(slide, mask=mask)
        >>> scores = module.score_model(slide, mask=mask)

    Parameters
    ----------
    in_features: int
        Features (model input) dimension.
    out_features: int
        Controls the number of scores and, by extension, the number of out_features.
    n_top: int
        Number of tiles with hightest scores that are selected and fed to the MLP.
    n_bottom: int
        Number of tiles with lowest scores that are selected and fed to the MLP.
    tiles_mlp_hidden: Optional[List[int]] = None
        Number of units for layers in the first MLP applied tile wise to compute
        a score for each tiles from the tile features.
        If `None`, a linear layer is used to compute tile scores.
        If e.g. `[128, 64]`, the tile scores are computed with a MLP of dimension
        features_dim -> 128 -> 64 -> 1.
    mlp_hidden: Optional[List[int]] = None
        Number of units for layers of the second MLP that combine top and bottom
        scores and outputs a final prediction at the slide-level. If `None`, a
        linear layer is used to compute the prediction from the extreme scores.
        If e.g. `[128, 64]`, the prediction is computed
        with a MLP n_top + n_bottom -> 128 -> 64 -> 1.
    mlp_dropout: Optional[List[float]] = None
        Dropout that is used for each layer of the MLP. If `None`, no dropout
        is used.
    mlp_activation: Optional[torch.nn.Module] = torch.nn.Sigmoid
        Activation that is used after each layer of the MLP.
    bias: bool = True
        Whether to add bias for layers of the tiles MLP.

    References
    ----------
    .. [1] Pierre Courtiol, Eric W. Tramel, Marc Sanselme, and Gilles Wainrib. Classification
    and disease localization in histopathology using only global labels: A weakly-supervised
    approach. CoRR, abs/1802.02212, 2018.
    """

    def __init__(
            self,
            in_features: int,
            out_features: int,
            n_top: Optional[int] = None,
            n_bottom: Optional[int] = None,
            tiles_mlp_hidden: Optional[List[int]] = None,
            mlp_hidden: Optional[List[int]] = None,
            mlp_dropout: Optional[List[float]] = None,
            mlp_activation: Optional[torch.nn.Module] = torch.nn.Sigmoid(),
            bias: bool = True,
    ) -> None:
        super(Chowder, self).__init__()
        if n_top is None and n_bottom is None:
            raise ValueError(
                "At least one of `n_top` or `n_bottom` must not be None."
            )

        if mlp_dropout is not None:
            if mlp_hidden is not None:
                assert len(mlp_hidden) == len(
                    mlp_dropout
                ), "mlp_hidden and mlp_dropout must have the same length"
            else:
                raise ValueError(
                    "mlp_hidden must have a value and have the same length as mlp_dropout if mlp_dropout is given."
                )

        self.score_model = TilesMLP(
            in_features,
            hidden=tiles_mlp_hidden,
            bias=bias,
            out_features=out_features,
        )
        self.score_model.apply(self.weight_initialization)

        self.extreme_layer = ExtremeLayer(n_top=n_top, n_bottom=n_bottom)

        mlp_in_features = n_top + n_bottom
        self.mlp = MLP(
            mlp_in_features,
            1,
            hidden=mlp_hidden,
            dropout=mlp_dropout,
            activation=mlp_activation,
        )
        self.mlp.apply(self.weight_initialization)

    @staticmethod
    def weight_initialization(module: torch.nn.Module) -> None:
        """Initialize weights for the module using Xavier initialization method,
        "Understanding the difficulty of training deep feedforward neural networks",
        Glorot, X. & Bengio, Y. (2010)."""
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.xavier_uniform_(module.weight)

            if module.bias is not None:
                module.bias.data.fill_(0.0)

    def forward(
            self, features: torch.Tensor, mask: Optional[torch.BoolTensor] = None
    ) -> torch.Tensor:
        """
        Parameters
        ----------
        features: torch.Tensor
            (B, N_TILES, IN_FEATURES)
        mask: Optional[torch.BoolTensor] = None
            (B, N_TILES, 1), True for values that were padded.

        Returns
        -------
        logits, extreme_scores: Tuple[torch.Tensor, torch.Tensor]:
            (B, OUT_FEATURES), (B, N_TOP + N_BOTTOM, OUT_FEATURES)
        """
        scores = self.score_model(x=features, mask=mask)
        extreme_scores = self.extreme_layer(
            x=scores, mask=mask
        )  # (B, N_TOP + N_BOTTOM, OUT_FEATURES)
#         print(extreme_scores.size())
        # Apply MLP to the N_TOP + N_BOTTOM scores.
        y = self.mlp(extreme_scores.transpose(1, 2))  # (B, OUT_FEATURES, 1)

        return y.squeeze(2)


In [2]:
!yes | sudo dpkg -i /kaggle/input/pyvips-install/libvips/*.deb
!pip install /kaggle/input/pyvips-install/pyvips/pyvips-2.2.1-py2.py3-none-any.whl --find-links=/kaggle/input/pyvips-install/pyvips/ --no-index



















































































































































































































































































































































































































































































































































































































yes: standard output: Broken pipe
Looking in links: /kaggle/input/pyvips-install/pyvips/
Processing /kaggle/input/pyvips-install/pyvips/pyvips-2.2.1-py2.py3-none-any.whl


In [3]:
import os
import pyvips
import random
os.environ["OPENCV_IO_MAX_IMAGE_PIXELS"] = pow(2,40).__str__()
import cv2
import rasterio
from rasterio.windows import Window
from tqdm import tqdm
import pandas as pd
import PIL.Image as Image
import numpy as np
import torch
from transformers import AutoImageProcessor, ViTModel
from io import BytesIO
import time
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from joblib.externals.loky.backend.context import get_context
# torch.multiprocessing.set_start_method('spawn')

os.environ['VIPS_CONCURRENCY'] = '4'
os.environ['VIPS_DISC_THRESHOLD'] = '15gb'

Image.MAX_IMAGE_PIXELS = 5000000000
random.seed(42)


ROOT_DIR = '/kaggle/input/UBC-OCEAN'
TEST_DIR = '/kaggle/input/UBC-OCEAN/test_thumbnails'
ALT_TEST_DIR = '/kaggle/input/UBC-OCEAN/test_images'
# TEST_DIR = '/kaggle/input/UBC-OCEAN/train_thumbnails'
# ALT_TEST_DIR = '/kaggle/input/UBC-OCEAN/train_images'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# weight_path = "/kaggle/input/modelstatedict/ckpt_0.870370.pth"
# weight_path = "/kaggle/input/modelstatedict/ckpt_0.521739.pth"
# weight_path = "/kaggle/input/modelstatedict/ckpt_0.907407.pth"

owkin_path = "/kaggle/input/modelstatedict/owkin/phikon"
# weight_path_list = ["ckpt_0.972119.pth", "ckpt_0.970260.pth", "ckpt_0.966543.pth", "ckpt_0.972119_t.pth",
#                  "ckpt_0.979554.pth", "ckpt_0.907407.pth", "ckpt_0.870370.pth", "best_0.918605.pth"]
weight_path_list = ["ckpt_0.972119.pth", "ckpt_0.970260.pth", "ckpt_0.966543.pth", "ckpt_0.972119_t.pth", 
                    "ckpt_0.979554.pth","ckpt_0.907407.pth","best_0.755814.pth"]
# weight_path_list = ["ckpt_0.972119.pth","ckpt_0.970260.pth","ckpt_0.966543.pth","ckpt_0.972119_t.pth",
#                  "ckpt_0.907407.pth","ckpt_0.870370.pth","best_0.918605.pth","best_0.755814.pth"]
# weight_path_list = ["/kaggle/input/modelstatedict/ckpt_0.907407.pth",
#                     "/kaggle/input/modelstatedict/ckpt_0.870370.pth", 
#                     "/kaggle/input/modelstatedict/best.pth",
#                     "/kaggle/input/modelstatedict/best_0.755814.pth"]


def get_wsl(image_id):
    if os.path.exists(f"{ALT_TEST_DIR}/{image_id}.png"):
        return f"{ALT_TEST_DIR}/{image_id}.png"
    else:
        return f"{TEST_DIR}/{image_id}_thumbnail.png"


def get_thumbnails(image_id):
    if os.path.exists(f"{TEST_DIR}/{image_id}_thumbnail.png"):
        return f"{TEST_DIR}/{image_id}_thumbnail.png"
    else:
        return f"{ALT_TEST_DIR}/{image_id}.png"


def tma_bbox(bbox):
    minx = 10e9
    miny = 10e9
    maxx = -1
    maxy = -1
    for x, y, w, h in bbox:
        minx = min(x, minx)
        miny = min(y, miny)
        maxx = max(x + w, maxx)
        maxy = max(y + h, maxy)
    bbox = [(minx, miny, maxx - minx, maxy - miny)]
    return bbox


def changeid2str(preds):
    str_list = []
    for i in range(preds.shape[0]):
        str_list.append(id2name[str(preds[i])])
    return str_list


def gen_maskbbox(path, id, is_tma):
    image = cv2.imread(path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    if is_tma:
        gray = 255 - gray
    ret, binary = cv2.threshold(gray, 60, 255, cv2.THRESH_BINARY)
    kernel = np.ones((21, 21), np.uint8)
    eroded = cv2.erode(binary, kernel, iterations=1)
    dilated = cv2.dilate(eroded, kernel, iterations=1)

    # 形态学开闭操作
    opened = cv2.morphologyEx(dilated, cv2.MORPH_OPEN, kernel)
    closed = cv2.morphologyEx(opened, cv2.MORPH_CLOSE, kernel)
    contours, hierarchy = cv2.findContours(closed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    bbox = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        bbox.append((x, y, w, h))
        # if not is_tma:
        #     cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    if is_tma:
        bbox = tma_bbox(bbox)
        # cv2.rectangle(image, (bbox[0][0], bbox[0][1]), (bbox[0][0] + bbox[0][2], bbox[0][1] + bbox[0][3]), (0, 255, 0),
        #               2)
    # cv2.imwrite("./data/rect/%s.png" % id, image)
    return bbox
    # cv2.imshow('Result', image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()


df = pd.read_csv("/kaggle/input/UBC-OCEAN/test.csv")
# df = pd.read_csv("/kaggle/input/UBC-OCEAN/train.csv")
df["file_path"] = df["image_id"].apply(get_wsl)
df["thumbnails"] = df["image_id"].apply(get_thumbnails)
name2id = {"CC": "0", "EC": "1", "HGSC": "2", "LGSC": "3", "MC": "4", "Other": "5"}
id2name = {v: k for k, v in name2id.items()}
patch_size = 256
stride = patch_size


image_processor = AutoImageProcessor.from_pretrained(owkin_path)
model = ViTModel.from_pretrained(owkin_path, add_pooling_layer=False)
model = model.eval().cuda()


chowder = Chowder(
    in_features=768,                     # output dimension of Phikon
    out_features=5,                      # dimension of predictions (a probability for class "1")
    n_top=5,                             # number of top scores in Chowder (in the image, N is 2)
    n_bottom=5,                          # number of bottom scores in Chowder
    mlp_hidden=[200, 100],               # MLP hidden layers after the max-min layer
    mlp_activation=torch.nn.Sigmoid(),   # MLP activation
    bias=True                            # bias for first 1D convolution which computes scores
)
# chowder.load_state_dict(torch.load(weight_path))
chowder = chowder.eval().cuda()



tiles_num = 256
df_sub = pd.DataFrame(columns=["image_id", "label"])


def get_top_10_percent_indices(data):
    # 获取前 10% 大的数据索引
    num_elements = len(data)
    num_top_elements = int(0.12 * num_elements)
#     print("num_top_elements",num_top_elements)
    
    # 使用 sorted 函数获取从大到小排序的数据索引
    sorted_indices = sorted(range(num_elements), key=lambda i: data[i], reverse=True)
    
    # 截取前 10% 大的数据索引
    top_10_percent_indices = sorted_indices[num_top_elements]
    
    return data[top_10_percent_indices]


def process_data(df_data):
    preds = {
        "image_id":[],
        "label":[],
        "H":[]
    }
    for idx, path in tqdm(enumerate(df_data["file_path"]), total=len(df_data["file_path"])):
        path_thumbnails = df_data["thumbnails"][idx]
        src = pyvips.Image.new_from_file(path)
        #     src = Image.open(path).convert("RGB")
        w, h = src.width, src.height
        thumbnails_img = Image.open(path_thumbnails).convert("RGB")
        wt, ht = thumbnails_img.size
        scale_factor = h / ht
        is_tma = (w == wt)
        bbox = gen_maskbbox(path_thumbnails, df_data["image_id"][idx], is_tma)
        # print("bbox ok ! ",len(bbox), scale_factor, " ", bbox)
        tiles = []
        for (bx, by, bw, bh) in bbox:
            if bw * bh < 10e4:
                continue
            sx, sy, sw, sh = round(bx * scale_factor), round(by * scale_factor), round(bw * scale_factor), round(
                bh * scale_factor)
            for j in range(0, (sh - patch_size) // stride + 1):
                for i in range(0, (sw - patch_size) // stride + 1):
                    tiles.append((sx + i * stride, sy + j * stride, 256, 256))
        random.shuffle(tiles)
        # print("shuffle ok ! ", len(tiles))
        cnt_max = 0
        feats = []
        for wds in tiles:
            if cnt_max < tiles_num:
                wx, wy, ww, wh = wds
                if (wx + ww) > (w - 1) or (wy + wh) > (h - 1):
                    continue
                crop_img = src.crop(wx, wy, ww, wh).numpy()
                crop_img = Image.fromarray(crop_img)
                crop_img_gray = crop_img.convert("L")
                crop_img_gray = np.array(crop_img_gray)
                mask = (crop_img_gray >= 230)
                crop_img_gray[mask] = 0
                if np.count_nonzero(crop_img_gray) >= patch_size * patch_size * 0.5:
                    cnt_max += 1
                    inputs = image_processor(crop_img, return_tensors="pt")
                    inputs["pixel_values"] = inputs["pixel_values"].cuda()

                    # get the features
                    with torch.no_grad():
                        outputs = model(**inputs)
                        features = outputs.last_hidden_state[:, 0, :]  # (1, 768) shape
                        feats.append(features)
        wsl_feat = torch.cat(feats, dim=0)
        if wsl_feat.size(0) < tiles_num:
            repeat_num = tiles_num // wsl_feat.size(0)
            pick_num = tiles_num % wsl_feat.size(0)
            picks = random.sample(range(0, wsl_feat.size(0)), pick_num)
            wsl_feat = torch.cat([wsl_feat.repeat(repeat_num, 1), wsl_feat[picks, :]], dim=0)
        with torch.no_grad():
#             outputs = chowder(wsl_feat.unsqueeze(0))
            pred = []
            for ckpt_path in weight_path_list:
                chowder.load_state_dict(torch.load("/kaggle/input/modelstatedict/"+ckpt_path))
                pred.append(torch.nn.Softmax(dim=1)(chowder(wsl_feat.unsqueeze(0))))
            pred = torch.sum(torch.cat(pred, dim=0),dim=0,keepdim=True)
#             _, predicted = torch.max(outputs, 1)
            _, predicted = torch.max(pred, 1)
            # pd.concat([df_sub, {"image_id":df_data["image_id"][idx],"label":id2name[str(int(predicted[0]))]}])
            # preds.append(predicted.detach().cpu().numpy())
            preds["image_id"].append(int(df_data["image_id"][idx]))
            H = -torch.sum(pred/len(weight_path_list) * torch.log(pred/len(weight_path_list)))
            preds["H"].append(float(H.detach().cpu().numpy()))
            preds["label"].append(id2name[str(int(predicted[0].cpu()))])
#             if H.detach().cpu().numpy() > 1.1:
#                 preds["label"].append("Other")
#             else:
#                 preds["label"].append(id2name[str(int(predicted[0].cpu()))])
    return pd.DataFrame.from_dict(preds)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Some weights of the model checkpoint at /kaggle/input/modelstatedict/owkin/phikon were not used when initializing ViTModel: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
from joblib import Parallel, delayed


df_parts = np.array_split(df, 2)

results = Parallel(n_jobs=2)(delayed(process_data)(part.reset_index(drop=True, inplace=False)) for part in df_parts)

# display(results)
# results["image_id"] = results["image_id"].astype(df["image_id"].dtype)
# display(results)
df_sub = pd.concat(results, ignore_index=True)
thr = get_top_10_percent_indices(df_sub["H"])
mask = df_sub["H"] > 1.3
df_sub["label"][mask] = "Other"
del df_sub["H"]

df_sub["image_id"] = df_sub["image_id"].astype(df["image_id"].dtype)
df_sub.to_csv("submission.csv", index=False)
display(df_sub)


/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
0it [00:00, ?it/s]
100%|██████████| 1/1 [00:33<00:00, 33.41s/it]
/tmp/ipykernel_26/3869971168.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

,image_id,label
0,41,HGSC
